In [38]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.models import Model
from keras.layers import Conv2D, Dense, BatchNormalization, Input
from keras.layers import Flatten, MaxPooling2D, Activation, RandomRotation
from keras.layers import AveragePooling2D, InputLayer, SpatialDropout2D 
from sklearn.metrics import confusion_matrix

In [39]:
data = np.load('flatland_train.npz')
X = data['X']
y = data['y'].astype(int)

y[y != 0] -= 2    # Correct labels so that triangle is mapped to class 1
X = X / 255.      # Scale down to range [0, 1]


# model.save('model.h5')

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

# Multiple different models

## First model
A couple of convolutional layers as well as a maxpooling layer.

In [42]:
def create_model():
  model = keras.models.Sequential()
  model.add(InputLayer(input_shape=[50, 50, 1]))

  model.add(Conv2D(128, kernel_size=(5, 5), strides=2))
  model.add(BatchNormalization())
  model.add(Activation("relu"))

  model.add(Conv2D(64, kernel_size=(3, 3)))
  model.add(BatchNormalization())
  model.add(Activation("relu"))

  model.add(MaxPooling2D(pool_size=(4, 4)))

  model.add(Conv2D(64, kernel_size=(3, 3)))
  model.add(BatchNormalization())
  model.add(Activation("relu"))

  model.add(Conv2D(64, kernel_size=(3, 3)))
  model.add(BatchNormalization())
  model.add(Activation("relu"))


  model.add(BatchNormalization())

  model.add(Flatten())
  model.add(Dense(1000, activation="relu"))
  model.add(keras.layers.Dropout(0.2))
  model.add(Dense(1000, activation="relu"))
  model.add(Dense(5, activation="softmax"))
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer='sgd',
                metrics=["accuracy"])

  return model

## Second model
Imitation of Lenet-5 architecture.

In [50]:
def create_model():
  model = keras.models.Sequential()
  model.add(InputLayer(input_shape=[50, 50, 1]))
  model.add(Conv2D(6, kernel_size=(5, 5)))
  model.add(BatchNormalization())
  model.add(Activation("tanh"))
  
  model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

  model.add(Conv2D(16, kernel_size=(5, 5)))
  model.add(BatchNormalization())
  model.add(Activation("tanh"))

  model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

  model.add(Conv2D(120, kernel_size=(5, 5)))
  model.add(BatchNormalization())
  model.add(Activation("tanh"))

  model.add(BatchNormalization())

  model.add(Flatten())
  model.add(Dense(84, activation="tanh"))
  model.add(keras.layers.Dropout(0.4))
  model.add(Dense(5, activation="softmax"))
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.Adam(3e-4),
                metrics=["accuracy"])

  return model

## Third model
Added RandomRotation layer and spatial dropout.

In [53]:
def create_model():
  model = keras.models.Sequential()
  model.add(InputLayer(input_shape=[50, 50, 1]))
  model.add(RandomRotation(factor=0.5, fill_mode='constant', fill_value=0.0))

  model.add(Conv2D(128, kernel_size=(5, 5), strides=2, activation='relu'))
  model.add(BatchNormalization())
  model.add(SpatialDropout2D(0.2))


  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(BatchNormalization())
  model.add(SpatialDropout2D(0.2))

  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(BatchNormalization())


  model.add(Flatten())
  model.add(Dense(1000, activation="relu"))
  model.add(keras.layers.Dropout(0.5))
  model.add(Dense(1000, activation="relu"))
  model.add(Dense(5, activation="softmax"))
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.Adam(learning_rate=3e-4),
                metrics=["accuracy"])

  return model

## Fourth model
Trying the dilation rate parameter.

In [ ]:
def create_model():
  model = keras.models.Sequential()
  model.add(InputLayer(input_shape=[50, 50, 1]))
  model.add(RandomRotation(factor=0.5, fill_mode='constant', fill_value=0.0))

  model.add(Conv2D(96, kernel_size=(5, 5), dilation_rate=2, activation='relu'))
  model.add(BatchNormalization())
  model.add(SpatialDropout2D(0.2))
  
  model.add(MaxPooling2D(pool_size=(3, 3), strides=3))

  model.add(Conv2D(128, kernel_size=(3, 3), dilation_rate=2, activation='relu'))
  model.add(BatchNormalization())
  model.add(SpatialDropout2D(0.1))
  
  model.add(MaxPooling2D(pool_size=(3, 3)))

  model.add(BatchNormalization())

  model.add(Flatten())
  model.add(Dense(1024, activation="relu"))
  model.add(keras.layers.Dropout(0.5))
  model.add(Dense(1024, activation="relu"))
  model.add(Dense(5, activation="softmax"))
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer='sgd',
                metrics=["accuracy"])

  return model

## Different representation of the third model
Reduced number of weights from layers.

In [ ]:
def create_model():
  inp = Input(shape=(50, 50, 1))

  out = RandomRotation(factor=0.5, fill_mode='constant', fill_value=0.0)(inp)
  out = Conv2D(64, kernel_size=(7, 7), strides=2, activation='relu')(out)
  out = BatchNormalization()(out)
  out = SpatialDropout2D(0.2)(out)

  out = Conv2D(64, kernel_size=(5, 5), activation='relu')(out)
  out = BatchNormalization()(out)
  out = SpatialDropout2D(0.2)(out)

  out = MaxPooling2D(pool_size=(2, 2))(out)

  out = Conv2D(64, kernel_size=(3, 3), activation='relu')(out)
  out = BatchNormalization()(out)

  out = MaxPooling2D(pool_size=(2, 2))(out)

  out = Conv2D(64, kernel_size=(3, 3), activation='relu')(out)
  out = BatchNormalization()(out)

  out = Flatten()(out)
  out = Dense(600, activation="relu")(out)
  out = keras.layers.Dropout(0.5)(out)
  out = Dense(5, activation="softmax")(out)
  
  model = Model(inputs=inp, outputs=out)
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.Adam(learning_rate=3e-4),
                metrics=["accuracy"])

  return model

## Best model (for now)
Switched some layers around.

In [28]:
def create_model():
  inp = Input(shape=(50, 50, 1))

  out = RandomRotation(factor=0.5, fill_mode='constant', fill_value=0.0)(inp)
  out = Conv2D(64, kernel_size=(7, 7), strides=2, activation='relu')(out)
  out = BatchNormalization()(out)
  out = SpatialDropout2D(0.2)(out)
  
  out = Conv2D(128, kernel_size=(5, 5), activation='relu')(out)
  out = BatchNormalization()(out)
  out = SpatialDropout2D(0.2)(out)

  out = MaxPooling2D(pool_size=(3, 3))(out)

  out = Conv2D(64, kernel_size=(3, 3), activation='relu')(out)
  out = BatchNormalization()(out)
  out = SpatialDropout2D(0.1)(out)

  out = Conv2D(32, kernel_size=(3, 3), activation='relu')(out)
  out = BatchNormalization()(out)

  out = Flatten()(out)
  out = Dense(512, activation="relu")(out)
  out = keras.layers.Dropout(0.5)(out)
  out = Dense(5, activation="softmax")(out)
  
  model = Model(inputs=inp, outputs=out)
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.Adam(learning_rate=3e-4),
                metrics=["accuracy"])

  return model

# Trials

In [ ]:
model = create_model()
model.summary()

In [55]:
loss = model.fit(X_train, y_train, batch_size=16, epochs=30, 
                 validation_data=(X_test, y_test))

Epoch 1/30
375/375 [==============================] - 4s 8ms/step - loss: 1.3825 - accuracy: 0.4090 - val_loss: 1.8269 - val_accuracy: 0.1447
Epoch 2/30
375/375 [==============================] - 3s 7ms/step - loss: 1.0128 - accuracy: 0.5690 - val_loss: 0.7823 - val_accuracy: 0.6525
Epoch 3/30
375/375 [==============================] - 3s 7ms/step - loss: 0.7563 - accuracy: 0.7003 - val_loss: 0.4189 - val_accuracy: 0.8717
Epoch 4/30
375/375 [==============================] - 2s 7ms/step - loss: 0.5866 - accuracy: 0.7878 - val_loss: 0.4475 - val_accuracy: 0.8438
Epoch 5/30
375/375 [==============================] - 3s 7ms/step - loss: 0.4961 - accuracy: 0.8262 - val_loss: 0.3471 - val_accuracy: 0.9028
Epoch 6/30
375/375 [==============================] - 3s 7ms/step - loss: 0.4158 - accuracy: 0.8730 - val_loss: 0.2106 - val_accuracy: 0.9625
Epoch 7/30
375/375 [==============================] - 3s 7ms/step - loss: 0.3768 - accuracy: 0.8810 - val_loss: 0.2026 - val_accuracy: 0.9670
Epoch 

In [ ]:
pd.DataFrame(loss.history).plot()

In [57]:
pred = model.predict(X_test).argmax(axis=1)
failures = X_test[pred != y_test]
index = np.arange(0, len(y_test))[pred != y_test]

125/125 [==============================] - 1s 3ms/step


In [58]:
failures.shape

(61, 50, 50)

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(12,16))

for i,ax in enumerate(axes.flat):
    ax.imshow(failures[i], cmap='gray')
    ax.set_axis_off()
    ax.set_title(f'P: {pred[index[i]]}, T: {y_test[index[i]]}')

# Transfer learning

In [ ]:
mobile_net = keras.applications.MobileNet(weights='imagenet', include_top=False,
                                          input_shape=(50, 50, 3))

mobile_net.trainable = False
    
inputs = keras.layers.Input(shape=(50, 50, 3))
embedding = mobile_net(inputs, training=False)
output = keras.layers.Flatten()(embedding)
output = keras.layers.Dense(5, activation='softmax')(output)
model = keras.models.Model(inputs=inputs, outputs=output)
model.summary()

model.compile(optimizer='sgd', loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [61]:
X_new_train = np.repeat(X_train[..., None], 3, axis=3)
X_new_test = np.repeat(X_test[..., None], 3, axis=3)

In [ ]:
model.fit(X_new_train, y_train, validation_data=(X_new_test, y_test), batch_size=16, 
          epochs=30)